In [ ]:
import time 
from network.network import NetworkManager

time_start = time.time()
network = NetworkManager()
graph = network.create_graph()
print(f"Czas inicjalizowania grafu: {time.time() - time_start}")

print(len(list(graph.nodes())))
graph.display()

In [ ]:
from unidecode import unidecode 
import os

path = os.path.join(os.path.abspath(os.getcwd()), "network_data")
print(path)
for file in os.listdir(path):
    full_path = os.path.join(path, file)
    new_file = unidecode(file).lower().replace(' ', '_')
    new_full_path = os.path.join(path, new_file)
    if file != new_file:
        os.rename(full_path, new_full_path)

In [ ]:
from models.simluation.engine import Simulation

sim = Simulation()

In [ ]:
sim.reset()
sim.inject_parameters(15, "day")
sim.run()

In [ ]:
import networkx as nx 
import os
import pickle
from network.graph_reader import GraphManager
from network.simulation_graph import SimulationGraph
import osmnx as ox

reader = GraphManager()


ox.settings.timeout = 10000 
ox.settings.log_console = True

filters = '["highway"~"motorway"]'

europe_countries = ["Francja"]

for country in europe_countries:
    graph = SimulationGraph.from_osmnx(country, custom_filter=filters)
    reader.save_pickle_file(f"{country}_motorway.pkl", graph)
ox.plot_graph(graph)

In [ ]:

from network.network import NetworkManager
import osmnx as ox


network = NetworkManager()
graph = network.create_graph()
graph.display((52.25,  52.20,  21.05, 21.00))

In [ ]:


from network.graph_reader import GraphManager

reader = GraphManager()
europe_graph = reader.load_pickle_graph("consolidated_europe_motorway.pkl")
graph = reader.load_pickle_graph("added_edges_motorway.pkl")


In [ ]:
import folium
from folium import FeatureGroup, PolyLine, CircleMarker
from IPython.display import display
from shapely.geometry import LineString, Point

def _node_lat_lon(node_data):
    """Pomocniczo: zwraca (lat, lon) z atrybutów noda (wspiera x/y, lon/lat, geometry)."""
    if node_data is None:
        return None
    # shapely Point
    geom = node_data.get("geometry")
    if isinstance(geom, Point):
        return (geom.y, geom.x)
    # common OSMnx: y = lat, x = lon
    if "y" in node_data and "x" in node_data:
        return (float(node_data["y"]), float(node_data["x"]))
    # alternative keys
    if "lat" in node_data and "lon" in node_data:
        return (float(node_data["lat"]), float(node_data["lon"]))
    if "lat" in node_data and "lng" in node_data:
        return (float(node_data["lat"]), float(node_data["lng"]))
    if "latitude" in node_data and "longitude" in node_data:
        return (float(node_data["latitude"]), float(node_data["longitude"]))
    # nothing found
    return None

def add_graph_routes_to_map(m: folium.Map, G, color="red", weight=3, opacity=0.8,
                            layer_name="Routes", show=True):
    """
    Dodaje wszystkie krawędzie grafu networkx jako czerwone trasy do mapy folium.

    Args:
        m: istniejący folium.Map (jeśli None — zostanie utworzona nowa mapa z centrum Europy).
        G: graf networkx (Graph lub MultiGraph). Oczekuje się współrzędnych w stopniach (lat/lon).
        color, weight, opacity: styl linii.
        layer_name: nazwa warstwy w mapie.
        show: jeśli True — display(map) (dla Jupyter).

    Returns:
        folium.Map
    """
    # utwórz mapę jeśli nie podano
    if m is None:
        m = folium.Map(location=(54.5260, 15.2551), zoom_start=4)

    fg = FeatureGroup(name=layer_name, show=True)

    # iterator zgodny z Graph i MultiGraph (krawędzie z attr dict)
    # dla MultiGraph: G.edges(keys=True, data=True)
    is_multigraph = getattr(G, "is_multigraph", lambda: False)()
    if is_multigraph:
        edges_iter = G.edges(keys=True, data=True)
    else:
        edges_iter = ((u, v, None, data) for u, v, data in G.edges(data=True))

    for item in edges_iter:
        if is_multigraph:
            u, v, k, data = item
        else:
            u, v, k, data = item
        
        if data.get("type") == "road":
            # jeśli edge ma geometry (LineString) - użyj jej
            geom = data.get("geometry") if isinstance(data, dict) else None
            if isinstance(geom, LineString):
                # shapely coords są (x=lon, y=lat)
                coords = [(float(lat), float(lon)) for lon, lat in geom.coords]
            else:
                # pobierz współrzędne nodów
                udata = G.nodes.get(u, {})
                vdata = G.nodes.get(v, {})
                u_latlon = _node_lat_lon(udata)
                v_latlon = _node_lat_lon(vdata)
                if u_latlon is None or v_latlon is None:
                    # brak współrzędnych — pomiń tę krawędź
                    continue
                coords = [u_latlon, v_latlon]

            # dodaj PolyLine
            PolyLine(
                locations=coords,
                color=color,
                weight=weight,
                opacity=opacity
            ).add_to(fg)

    fg.add_to(m)
    folium.LayerControl().add_to(m)

    if show:
        display(m)
    return m


def add_graph_airline_routes_to_map(m: folium.Map, G, color="red", weight=3, opacity=0.8,
                            layer_name="Routes", show=True):
    """
    Dodaje wszystkie krawędzie grafu networkx jako czerwone trasy do mapy folium.

    Args:
        m: istniejący folium.Map (jeśli None — zostanie utworzona nowa mapa z centrum Europy).
        G: graf networkx (Graph lub MultiGraph). Oczekuje się współrzędnych w stopniach (lat/lon).
        color, weight, opacity: styl linii.
        layer_name: nazwa warstwy w mapie.
        show: jeśli True — display(map) (dla Jupyter).

    Returns:
        folium.Map
    """
    # utwórz mapę jeśli nie podano
    if m is None:
        m = folium.Map(location=(54.5260, 15.2551), zoom_start=4)

    fg = FeatureGroup(name=layer_name, show=True)

    # iterator zgodny z Graph i MultiGraph (krawędzie z attr dict)
    # dla MultiGraph: G.edges(keys=True, data=True)
    is_multigraph = getattr(G, "is_multigraph", lambda: False)()
    if is_multigraph:
        edges_iter = G.edges(keys=True, data=True)
    else:
        edges_iter = ((u, v, None, data) for u, v, data in G.edges(data=True))

    for item in edges_iter:
        if is_multigraph:
            u, v, k, data = item
        else:
            u, v, k, data = item
        
        if data.get("type") == "airline_route":
            # jeśli edge ma geometry (LineString) - użyj jej
            geom = data.get("geometry") if isinstance(data, dict) else None
            if isinstance(geom, LineString):
                # shapely coords są (x=lon, y=lat)
                coords = [(float(lat), float(lon)) for lon, lat in geom.coords]
            else:
                # pobierz współrzędne nodów
                udata = G.nodes.get(u, {})
                vdata = G.nodes.get(v, {})
                u_latlon = _node_lat_lon(udata)
                v_latlon = _node_lat_lon(vdata)
                if u_latlon is None or v_latlon is None:
                    # brak współrzędnych — pomiń tę krawędź
                    continue
                coords = [u_latlon, v_latlon]

            # dodaj PolyLine
            PolyLine(
                locations=coords,
                color=color,
                weight=weight,
                opacity=opacity
            ).add_to(fg)

    fg.add_to(m)
    folium.LayerControl().add_to(m)

    if show:
        display(m)
    return m


def add_graph_nodes_to_map(m: folium.Map, G, color="blue", radius=3, opacity=0.8,
                           layer_name="Nodes", show=True):
    """
    Dodaje wszystkie węzły grafu networkx jako punkty (CircleMarker) do mapy folium.

    Args:
        m: istniejący folium.Map (jeśli None — zostanie utworzona nowa mapa z centrum Europy).
        G: graf networkx (Graph lub MultiGraph). Oczekuje współrzędnych w stopniach (lat/lon).
        color: kolor markerów.
        radius: promień markerów w pikselach.
        opacity: przezroczystość markerów.
        layer_name: nazwa warstwy w mapie.
        show: jeśli True — display(map) (dla Jupyter).

    Returns:
        folium.Map
    """
    if m is None:
        m = folium.Map(location=(54.5260, 15.2551), zoom_start=4)

    fg = FeatureGroup(name=layer_name, show=True)

    for node, data in G.nodes(data=True):
        latlon = _node_lat_lon(data)
        if latlon is None:
            continue  # pomiń węzeł bez współrzędnych
        CircleMarker(
            location=latlon,
            radius=radius,
            color=color,
            fill=True,
            fill_opacity=opacity
        ).add_to(fg)

    fg.add_to(m)
    folium.LayerControl().add_to(m)

    if show:
        display(m)
    return m


def add_path_to_map(m: folium.Map, G, path, color="red", weight=3, opacity=0.8,
                            layer_name="Route", show=True):
    """
    Rysuje na mapie folium ścieżkę zdefiniowaną przez listę ID węzłów (path).

    Args:
        m: istniejący folium.Map (jeśli None — zostanie utworzona nowa mapa).
        G: graf networkx (Graph lub MultiGraph).
        path: lista ID węzłów (np. [123, 456, 789]).
        color, weight, opacity: styl linii.
        layer_name: nazwa warstwy.
        show: czy wyświetlić mapę.

    Returns:
        folium.Map
    """
    # 1. Sprawdzenie czy path jest poprawna
    if not path or len(path) < 2:
        print("Ścieżka jest za krótka do wyświetlenia.")
        return m

    # 2. Utwórz mapę jeśli nie podano (centrowanie na pierwszym punkcie ścieżki)
    if m is None:
        start_node = G.nodes[path[0]]
        start_lat = start_node.get('y', start_node.get('lat'))
        start_lon = start_node.get('x', start_node.get('lon'))
        m = folium.Map(location=(start_lat, start_lon), zoom_start=13)

    fg = FeatureGroup(name=layer_name, show=True)
    
    # Pomocnicza funkcja do wyciągania lat/lon z węzła
    def _get_node_coords(node_id):
        n = G.nodes[node_id]
        # OSMnx używa 'y'/'x', inne standardy 'lat'/'lon'
        lat = n.get('y', n.get('lat'))
        lon = n.get('x', n.get('lon'))
        return (float(lat), float(lon)) if lat is not None and lon is not None else None

    is_multigraph = getattr(G, "is_multigraph", lambda: False)()

    # 3. Iteracja po parach węzłów w ścieżce: (u, v)
    for i in range(len(path) - 1):
        u = path[i]
        v = path[i + 1]

        # Sprawdź czy krawędź istnieje w grafie
        if not G.has_edge(u, v):
            continue

        edge_data = {}
        
        # Pobranie danych krawędzi
        if is_multigraph:
            # W MultiGraph między u i v może być wiele krawędzi (np. różne klucze 0, 1)
            # Pobieramy wszystkie i wybieramy tę najkrótszą (najbardziej prawdopodobną dla routingu)
            edges_dict = G.get_edge_data(u, v)
            # Wybieramy krawędź o najmniejszej długości (jeśli atrybut length istnieje), w przeciwnym razie pierwszą
            try:
                best_key = min(edges_dict, key=lambda k: edges_dict[k].get('length', float('inf')))
                edge_data = edges_dict[best_key]
            except (ValueError, TypeError):
                # Fallback: bierzemy pierwszą dostępną
                edge_data = list(edges_dict.values())[0]
        else:
            edge_data = G.get_edge_data(u, v)

        # 4. Obsługa geometrii (dokładny przebieg drogi vs linia prosta)
        geom = edge_data.get("geometry")
        
        coords = []
        if isinstance(geom, LineString):
            # shapely coords są (x=lon, y=lat), folium potrzebuje (lat, lon)
            coords = [(float(lat), float(lon)) for lon, lat in geom.coords]
        else:
            # Jeśli brak geometrii, rysujemy prostą linię między węzłami
            u_coords = _get_node_coords(u)
            v_coords = _get_node_coords(v)
            if u_coords and v_coords:
                coords = [u_coords, v_coords]

        if coords:
            PolyLine(
                locations=coords,
                color=color,
                weight=weight,
                opacity=opacity,
                tooltip=f"Edge: {u}->{v}"
            ).add_to(fg)

    fg.add_to(m)
    
    # Dodaj kontrolę warstw tylko jeśli jeszcze jej nie ma (żeby nie dublować przy wielokrotnym wywołaniu)
    # Prosty hack: sprawdzamy czy w _children mapy jest coś z nazwą 'leaflet-control-layers'
    # Ale dla bezpieczeństwa w Jupyterze zazwyczaj dodaje się na końcu.
    # Tutaj zostawiam standardowo:
    folium.LayerControl().add_to(m)

    if show:
        display(m)
    return m

def add_graph_routes_to_map2(m, graph, color="red", weight=2, opacity=0.9, layer_name="All edges"):
    """
    Rysuje trasy z grafu NetworkX na mapie Folium.
    Zakłada, że krawędzie mają atrybut 'linestring' z obiektem Shapely LineString.
    """
    # Tworzymy grupę (warstwę), aby można ją było włączać/wyłączać w panelu warstw
    feature_group = folium.FeatureGroup(name=layer_name)

    # Iteracja po wszystkich krawędziach grafu wraz z ich atrybutami
    for u, v, data in graph.edges(data=True):
        
        # Sprawdzamy, czy krawędź posiada atrybut 'linestring'
        if 'linestring' in data:
            geom = data['linestring']
            
            # Shapely przechowuje współrzędne jako (Długość, Szerokość) -> (x, y)
            # Folium wymaga formatu (Szerokość, Długość) -> (lat, lon)
            # Musimy zamienić kolejność współrzędnych:
            if hasattr(geom, 'coords'):
                locations = [(y, x) for x, y in geom.coords]
            else:
                # Fallback: jeśli linestring jest zwykłą listą punktów [(lat, lon), ...]
                locations = geom

            # Dodajemy linię do grupy
            folium.PolyLine(
                locations=locations,
                color=color,
                weight=weight,
                opacity=opacity
            ).add_to(feature_group)

    # Dodajemy całą grupę do mapy
    feature_group.add_to(m)

    # Dodajemy kontroler warstw (jeśli jeszcze go nie ma, Folium poradzi sobie z duplikatami w renderingu,
    # ale dobrą praktyką jest dodawanie go na samym końcu budowania mapy)
    folium.LayerControl().add_to(m)

    return m


In [ ]:
import folium

# cały graf
m = folium.Map(location=(52.0, 10.0), zoom_start=5)  # centrum Europy/bliski region
m = add_graph_routes_to_map(m, graph, color="red", weight=2, opacity=0.9, layer_name="All edges")

m

In [ ]:
import folium

# połączenia lotnicze 
m = folium.Map(location=(52.0, 10.0), zoom_start=5)  # centrum Europy/bliski region
m = add_graph_airline_routes_to_map(m, graph, color="red", weight=2, opacity=0.9, layer_name="All edges")

m

In [ ]:
import folium

# cały graf z linestringa
m = folium.Map(location=(52.0, 10.0), zoom_start=5)  # centrum Europy/bliski region
m = add_graph_routes_to_map2(m, graph, color="red", weight=2, opacity=0.9, layer_name="All edges")

m

In [ ]:
import folium

# połączone nody
m = folium.Map(location=(52.0, 10.0), zoom_start=5)  # centrum Europy/bliski region
add_graph_nodes_to_map(m, graph, color="green", radius=4)

In [ ]:

from network.graph_reader import GraphManager

reader = GraphManager()

europe_network = reader.load_pickle_graph("consolidated_europe_motorway.pkl")
empty_graph = europe_network.coherence(threshold=200)

for node, data in empty_graph.nodes(data=True):
    print(data)

empty_graph.nodes(data=True)[6190]

In [ ]:
from network.network import NetworkManager
from network.graph_reader import GraphManager
import osmnx as ox
import time

time_start = time.time()
network = NetworkManager()
reader = GraphManager()


graph = reader.load_pickle_graph("europe_motorway.pkl")
print(f"Czas inicjalizowania grafu: {time.time() - time_start}")
node1 = graph.get_nearest_node(52, 21)
node2 = graph.get_nearest_node(52.5, 13)

node_milano = graph.get_nearest_node(45.46, 9.19)
node_hamburg = graph.get_nearest_node(53.5, 10.0)

astar_time = time.time()
path1 = graph.astar(node1, node2)
path2 = graph.astar(node_milano, node_hamburg)
print(f"Liczba nodów: {len(path2)}, czas wykonywania astara: {round(time.time() - astar_time, 2)}")

In [ ]:
import osmnx as ox
from network.graph_reader import GraphManager
from network.simulation_graph import SimulationGraph

reader = GraphManager()
filter = '["aeroway"]'

graph = reader.load_pickle_graph("europe_motorway.pkl")

europe = graph.get_border_polygon()
print(europe)
# airports_graph = ox.graph_from_polygon(europe, custom_filter=filter)

# print(len(list(airports_graph.nodes())))
# sim = SimulationGraph(incoming_graph_data=airports_graph, default_capacity=10, default_price=100, type="airport")
# reader.save_pickle_file("europe_airports.pkl", sim)

In [ ]:
from network.network import NetworkManager

network = NetworkManager()
graph = network.load_seaports_graph(10, 100)
for u, v, n, data in graph.edges(data=True, keys=True):
    print(data)

for u, data in graph.nodes(data=True):
    print(data) 

In [ ]:
from network.network import NetworkManager
import time

network = NetworkManager()
# graph = network.load_seaports_graph(10, 100)
# for n, data in graph.nodes(data=True):
#     if "IATA" not in data:
#         print(data)

graph = network.get_graph_from_file("world", road_type="test")
# airplane_graph = network.load_airports_graph(default_capacity=10, default_price=7)
# graph.compose(airplane_graph)
# seaport_graph = network.load_seaports_graph(default_capacity=5, default_price=1)
# graph.compose(seaport_graph)
# graph.connect_airports_seaports(default_capacity=1000, default_price=0.5)

# Warszawa
node1 = graph.get_nearest_node(52, 21)

# Berlin
node2 = graph.get_nearest_node(52.5, 13)

# Stockholm
node3 = graph.get_nearest_node(59.32, 18.06)

# Londyn
node4 = graph.get_nearest_node(51.50, 0.12)

astar_time = time.time()
path_length = graph.astar(node2, node4)
print(f"Liczba nodów: {len(path_length)}, czas wykonywania astara: {round(time.time() - astar_time, 2)}")
print(path_length)

path_cost = graph.astar(node2, node4, metric="cost")
print(f"Liczba nodów: {len(path_cost)}, czas wykonywania astara: {round(time.time() - astar_time, 2)}")
print(path_cost)
graph = network.get_graph_from_file("added_world_edges", road_type="motorway")


In [ ]:
from network.network import NetworkManager
import time

network = NetworkManager()

graph = network.get_graph_from_file("added_world_edges", road_type="motorway")

In [ ]:
import folium

# path
m = folium.Map(location=(52.0, 10.0), zoom_start=5)  # centrum Europy/bliski region
add_path_to_map(m, graph, path_cost, color="green")

In [ ]:
from network.network import NetworkManager
import time

network = NetworkManager()
# graph = network.load_seaports_graph(10, 100)
# for n, data in graph.nodes(data=True):
#     if "IATA" not in data:
#         print(data)

graph = network.get_graph_from_file("europe")
airplane_graph = network.load_airports_graph(default_capacity=10, default_price=7)
graph.compose(airplane_graph)
seaport_graph = network.load_seaports_graph(default_capacity=5, default_price=1)
graph.compose(seaport_graph)
graph.connect_airports_seaports(default_capacity=1000, default_price=0.5)
for u, v, k, data in graph.edges(data=True, keys=True):
    max_speed = data.get("maxspeed")
    type = data.get("type")
    if type not in ("road", "sea_route", "airline_route"):
        print(data)


In [ ]:
from network.simulation_graph import SimulationGraph
from network.world import world_top_cities
from network.graph_reader import GraphManager

reader = GraphManager()

countries = []
for country in world_top_cities.keys():
    for city in world_top_cities[country]:
        graph = SimulationGraph.from_osmnx(city)
        reader.save_pickle_file(f"{city.replace(" ", "_").replace(",", "").lower()}_motorway.pkl", graph)
        print(city)